In [1]:
import cv2
import numpy as np
import dlib
from math import hypot #hypot is used to find euclidean distance betwen two coordinates

# Loading Camera and Nose image and Creating mask
cap = cv2.VideoCapture(0)
nose_image = cv2.imread("nose.png")
glow = cv2.imread("glow5.png")
mouth_image = cv2.imread("mouth.png")
_, frame = cap.read()
rows, cols, _ = frame.shape
mask = np.zeros((rows, cols), np.uint8)

# Loading Face detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

while True:
    _, frame = cap.read()
    #mask.fill(0)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(frame)
    for face in faces:
        landmarks = predictor(gray_frame, face)
        
        #Eye coordinates
        #Left eye:
        edge1 = (landmarks.part(43).x, landmarks.part(43).y)
        edge2 = (landmarks.part(44).x, landmarks.part(44).y)
        
        eye_width = int(hypot(edge1[0] - edge2[0], edge1[1] - edge2[1]))
        eye_coordinates = (edge1[0]+eye_width//2, edge1[1]+eye_width//2)
       
        try:
            #Adding the glow on eye
            eye_glow = cv2.resize(glow, (eye_width, eye_width))
            eye_glow_gray = cv2.cvtColor(eye_glow, cv2.COLOR_BGR2GRAY)
            _, mask = cv2.threshold(eye_glow_gray, 25, 255, cv2.THRESH_BINARY_INV)

            eye_area = frame[edge1[1] : edge1[1]+ eye_width, edge1[0] : edge1[0]+ eye_width] 
            eye_area_no_eye = cv2.bitwise_and(eye_area, eye_area, mask=mask) #Adding the eyes using bitwise addition
            final_eye = cv2.add(eye_area_no_eye, eye_glow)
    
            frame[edge1[1] : edge1[1]+ eye_width, edge1[0] : edge1[0]+ eye_width] = final_eye
        except:
            pass
        
        #Right eye:
        edge1 = (landmarks.part(37).x, landmarks.part(37).y)
        edge2 = (landmarks.part(38).x, landmarks.part(38).y)
        
        eye_width = int(hypot(edge1[0] - edge2[0],edge1[1] - edge2[1]))
        eye_coordinates = (edge1[0]+eye_width//2, edge1[1]+eye_width//2)
        
        try:
            #Adding the glow on eye
            eye_glow = cv2.resize(glow, (eye_width, eye_width))
            eye_glow_gray = cv2.cvtColor(eye_glow, cv2.COLOR_BGR2GRAY)
            _, mask = cv2.threshold(eye_glow_gray, 25, 255, cv2.THRESH_BINARY_INV)

            eye_area = frame[edge1[1] : edge1[1]+ eye_width, edge1[0] : edge1[0]+ eye_width] 
            eye_area_no_eye = cv2.bitwise_and(eye_area, eye_area, mask=mask) #Adding the eyes using bitwise addition
            final_eye = cv2.add(eye_area_no_eye, eye_glow)

            frame[edge1[1] : edge1[1]+ eye_width, edge1[0] : edge1[0]+ eye_width] = final_eye

        except:
            pass
       
    
        #Mouth coordinates
        mouth_edge1 = (landmarks.part(2).x, landmarks.part(2).y)
        mouth_edge2 = (landmarks.part(14).x, landmarks.part(14).y)
        mouth_bottom = (landmarks.part(8).x, landmarks.part(8).y)
        mouth_top = (landmarks.part(51).x, landmarks.part(51).y)
        mouth_centre = (landmarks.part(62).x, landmarks.part(62).y)
        
        mouth_width = int(hypot(mouth_edge2[0]-mouth_edge1[0], mouth_edge2[1]-mouth_edge1[1]))+10 
        mouth_height = int(hypot(mouth_top[0]-mouth_bottom[0], mouth_top[1]-mouth_bottom[1]))*2+50
        #The changes I have made to the calculated values of height and width here are by trial and error

        #New mouth position
        top_left = (int(mouth_centre[0]-mouth_width/2), int(mouth_centre[1]-mouth_height/2))
        bottom_right = (int(mouth_centre[0]+mouth_width/2), int(mouth_centre[1]+mouth_height))
        
        
        try:
            #Adding mouth
            mouth = cv2.resize(mouth_image, (mouth_width, mouth_height))
            mouth_gray = cv2.cvtColor(mouth, cv2.COLOR_BGR2GRAY)
            _, mask = cv2.threshold(mouth_gray, 25, 255, cv2.THRESH_BINARY_INV)
    
            mouth_area = frame[top_left[1]: top_left[1] + mouth_height, top_left[0]: top_left[0] + mouth_width]
            mouth_area_no_mouth = cv2.bitwise_and(mouth_area, mouth_area, mask=mask) #Adding the mouth using bitwise addition
            final_mouth = cv2.add(mouth_area_no_mouth, mouth)
    
            frame[top_left[1]: top_left[1] + mouth_height, top_left[0]: top_left[0] + mouth_width] = final_mouth
        except:
            pass
        
        
         # Nose coordinates
        top_nose = (landmarks.part(29).x, landmarks.part(29).y)
        center_nose = (landmarks.part(30).x, landmarks.part(30).y)
        left_nose = (landmarks.part(31).x, landmarks.part(31).y)
        right_nose = (landmarks.part(35).x, landmarks.part(35).y)

        nose_width = int(hypot(left_nose[0] - right_nose[0], left_nose[1] - right_nose[1]) * 1.7)
        nose_height = int(nose_width * 0.77)
   
        # New nose position
        top_left = (int(center_nose[0] - nose_width / 2), int(center_nose[1] - nose_height / 2))
        bottom_right = (int(center_nose[0] + nose_width / 2), int(center_nose[1] + nose_height / 2))
        
        try:
            # Adding the new nose
            nose_new = cv2.resize(nose_image, (nose_width, nose_height))
            nose_new_gray = cv2.cvtColor(nose_new, cv2.COLOR_BGR2GRAY)
            _, mask = cv2.threshold(nose_new_gray, 25, 255, cv2.THRESH_BINARY_INV)
    
            nose_area = frame[top_left[1]: top_left[1] + nose_height, top_left[0]: top_left[0] + nose_width]
            nose_area_no_nose = cv2.bitwise_and(nose_area, nose_area, mask=mask) #Adding the nose using bitwise addition
            final_nose = cv2.add(nose_area_no_nose, nose_new)
    
            frame[top_left[1]: top_left[1] + nose_height, top_left[0]: top_left[0] + nose_width] = final_nose
        except:
            pass
        
    cv2.imshow("Frame", frame)

    '''Note: The reason for using try is to avoid the code from crashing when coordinates of a point needed for placing
       the clip art becomes out of the given area of the camera feed'''


    if  cv2.waitKey(1) >= 0: #Press any to trigger
        cv2.destroyAllWindows()
        cap.release()
        break